In [1]:
import os
import cv2
import math
import numpy as np
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from scipy import stats as s

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [101]:
# open the .txt file which have names of training videos
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.tail()

,video_name
863,BenchPress/v_BenchPress_g23_c01.avi 10
864,BenchPress/v_BenchPress_g23_c02.avi 10
865,BenchPress/v_BenchPress_g23_c03.avi 10
866,BenchPress/v_BenchPress_g23_c04.avi 10
867,BenchPress/v_BenchPress_g23_c05.avi 10


In [102]:
# open the .txt file which have names of validation videos
f = open("vallist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
val = pd.DataFrame()
val['video_name'] = videos
val = val[:-1]
val.tail()

,video_name
284,BenchPress/v_BenchPress_g25_c03.avi
285,BenchPress/v_BenchPress_g25_c04.avi
286,BenchPress/v_BenchPress_g25_c05.avi
287,BenchPress/v_BenchPress_g25_c06.avi
288,BenchPress/v_BenchPress_g25_c07.avi


In [103]:
# open the .txt file which have names of test videos
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.tail()

,video_name
212,BenchPress/v_BenchPress_g04_c03.avi
213,BenchPress/v_BenchPress_g04_c04.avi
214,BenchPress/v_BenchPress_g04_c05.avi
215,BenchPress/v_BenchPress_g04_c06.avi
216,BenchPress/v_BenchPress_g04_c07.avi


In [104]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

train.tail()

,video_name,tag
863,BenchPress/v_BenchPress_g23_c01.avi 10,BenchPress
864,BenchPress/v_BenchPress_g23_c02.avi 10,BenchPress
865,BenchPress/v_BenchPress_g23_c03.avi 10,BenchPress
866,BenchPress/v_BenchPress_g23_c04.avi 10,BenchPress
867,BenchPress/v_BenchPress_g23_c05.avi 10,BenchPress


In [105]:
base_dir = 'data'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)

In [106]:
dir_names = set(train_video_tag)
print(len(dir_names))
print(dir_names)

10
{'BalanceBeam', 'Basketball', 'ApplyLipstick', 'BenchPress', 'BabyCrawling', 'Archery', 'BaseballPitch', 'BasketballDunk', 'BandMarching', 'ApplyEyeMakeup'}


In [107]:
for dir in dir_names:
    train_vids_dir = os.path.join(train_dir, dir)
    val_vids_dir = os.path.join(val_dir, dir)
    test_vids_dir = os.path.join(test_dir, dir)
    
    os.makedirs(train_vids_dir)
    os.makedirs(val_vids_dir)
    os.makedirs(test_vids_dir)

In [108]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    #cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0])
    frameRate = cap.get(5) #frame rate
    
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename = train_dir + '/' + videoFile.split('/')[0] + '/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count
            frame = cv2.resize(frame, (128, 128))
            cv2.imwrite(filename, frame)
            count+=1
    cap.release()

100%|██████████| 868/868 [00:35<00:00, 24.70it/s]


In [109]:
# storing the frames from validation videos
for i in tqdm(range(val.shape[0])):
    count = 0
    videoFile = val['video_name'][i]
    #cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0])
    frameRate = cap.get(5) #frame rate
    
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename = val_dir + '/' + videoFile.split('/')[0] + '/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count
            frame = cv2.resize(frame, (128, 128))
            cv2.imwrite(filename, frame)
            count+=1
    cap.release()

100%|██████████| 289/289 [00:13<00:00, 21.34it/s]


In [196]:
train_image = []
train_class = []

for dir_id in tqdm(range(len(dir_names))):
    images = glob('data/train/' + list(dir_names)[dir_id] + '/*.jpg')
    for i in range(len(images)):
        train_image.append(images[i].split('/')[3])
        train_class.append(images[i].split('/')[3].split('_')[1])
        
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

train_data.to_csv('UCF/train_new.csv',header=True, index=False)

100%|██████████| 10/10 [00:00<00:00, 236.49it/s]


In [197]:
train = pd.read_csv('UCF/train_new.csv')
train.tail()

,image,class
5350,v_ApplyEyeMakeup_g15_c03.avi_frame0.jpg,ApplyEyeMakeup
5351,v_ApplyEyeMakeup_g22_c03.avi_frame0.jpg,ApplyEyeMakeup
5352,v_ApplyEyeMakeup_g10_c03.avi_frame1.jpg,ApplyEyeMakeup
5353,v_ApplyEyeMakeup_g09_c02.avi_frame8.jpg,ApplyEyeMakeup
5354,v_ApplyEyeMakeup_g15_c04.avi_frame3.jpg,ApplyEyeMakeup


In [198]:
y_tr = train['class']
y_tr_dummy = pd.get_dummies(y_tr)

In [199]:
val_image = []
val_class = []

for dir_id in tqdm(range(len(dir_names))):
    images = glob('data/validation/' + list(dir_names)[dir_id] + '/*.jpg')
    for i in range(len(images)):
        val_image.append(images[i].split('/')[3])
        val_class.append(images[i].split('/')[3].split('_')[1])
        
val_data = pd.DataFrame()
val_data['image'] = val_image
val_data['class'] = val_class

val_data.to_csv('UCF/val_new.csv',header=True, index=False)

100%|██████████| 10/10 [00:00<00:00, 662.40it/s]


In [200]:
val = pd.read_csv('UCF/val_new.csv')
val.tail()

,image,class
1714,v_ApplyEyeMakeup_g05_c01.avi_frame6.jpg,ApplyEyeMakeup
1715,v_ApplyEyeMakeup_g06_c06.avi_frame1.jpg,ApplyEyeMakeup
1716,v_ApplyEyeMakeup_g24_c05.avi_frame2.jpg,ApplyEyeMakeup
1717,v_ApplyEyeMakeup_g25_c02.avi_frame1.jpg,ApplyEyeMakeup
1718,v_ApplyEyeMakeup_g07_c01.avi_frame3.jpg,ApplyEyeMakeup


In [201]:
print(len(train))
print(len(val))

5355
1719


In [202]:
y_val = val['class']
y_val_dummy = pd.get_dummies(y_val)

In [203]:
#base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128,128,3), pooling='avg')
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))

In [204]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [205]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 64

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512)) #like in block5_pool
    labels = np.zeros(shape=(sample_count, 10))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical')
    
    i = 0
    
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    
    return features, labels

In [206]:
train_features, train_labels = extract_features(train_dir, len(train))

Found 5355 images belonging to 10 classes.


In [207]:
validation_features, validation_labels = extract_features(validation_dir, len(val))

Found 1719 images belonging to 10 classes.


In [208]:
print(train_features.shape)
print(validation_features.shape)

(5355, 4, 4, 512)
(1719, 4, 4, 512)


In [209]:
import sys
print(sys.getsizeof(train_features)*1e-6)
print(sys.getsizeof(validation_features)*1e-6)

350.945424
112.656528


In [210]:
# flatten them to (samples, 8192):
train_features = np.reshape(train_features, (5355, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1719, 4 * 4 * 512))

In [211]:
print(train_features.shape)
print(validation_features.shape)

(5355, 8192)
(1719, 8192)


In [254]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [255]:
from keras.callbacks import ModelCheckpoint
save_weights = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [256]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [257]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 2,099,978
Trainable params: 2,099,978
Non-trainable params: 0
_________________________________________________________________


In [258]:
history = model.fit(train_features, train_labels,
                    epochs=5,
                    batch_size=128,
                    callbacks=[save_weights],
                    validation_data=(validation_features, validation_labels))

Train on 5355 samples, validate on 1719 samples
Epoch 1/5
5355/5355 [==============================] - 6s 1ms/step - loss: 0.7696 - acc: 0.7617 - val_loss: 0.7709 - val_acc: 0.7487
Epoch 2/5
5355/5355 [==============================] - 4s 747us/step - loss: 0.1041 - acc: 0.9770 - val_loss: 0.8699 - val_acc: 0.7400
Epoch 3/5
5355/5355 [==============================] - 4s 739us/step - loss: 0.0481 - acc: 0.9925 - val_loss: 0.8197 - val_acc: 0.7696
Epoch 4/5
5355/5355 [==============================] - 4s 772us/step - loss: 0.0295 - acc: 0.9955 - val_loss: 0.8565 - val_acc: 0.7493
Epoch 5/5
5355/5355 [==============================] - 4s 791us/step - loss: 0.0210 - acc: 0.9970 - val_loss: 0.8509 - val_acc: 0.7528


In [259]:
model.load_weights("weight.hdf5")

In [260]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [261]:
f = open("testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi


In [268]:
train = pd.read_csv('UCF/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [269]:
predict = []
actual = []

for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    #cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    cap = cv2.VideoCapture('UCF/'+videoFile.split(' ')[0])
    
    frameRate = cap.get(5) #frame rate
    x=1
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(128,128,3))
        img = image.img_to_array(img)
        prediction_images.append(img)
        
    prediction_images = np.array(prediction_images)
    prediction_images = conv_base.predict(prediction_images)
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 4 * 4 * 512)
    prediction = model.predict_classes(prediction_images)
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    actual.append(videoFile.split('/')[1].split('_')[1])

100%|██████████| 217/217 [03:47<00:00,  1.05s/it]


In [270]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

58.986175115207374